<div style="text-align: center; font-weight: bold; font-size: 300%">Rendu MAP412</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">École Polytechnique, MAP412 - septembre 2019</div><br />  
<div style="text-align: center; font-size: 120%">Jean-Charles Layoun</div>

In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

output_notebook(hide_banner=True)

**Indiquez le temps de préparation de ce notebook :**
<br>1h pour finir Runge-kutta 2 et 4 et leur convergence et l'oscillateur harmonique méthode euler<br>
2h pour faire le reste sans le bonus, erreur dans le corriger m'a fait faire un debugging inutile pdt 1h...<br>
45 min pour le bonus, questions théoriques et pratiques.<br>
1h relecture et mise en page.

# Méthode de Runge-Kutta

## La méthode RK2

**Implémenter la méthode RK2. Tester le cas $f(t,y)=y$, pour $t_0=0$, $y_0=1$ et $T=1,$.**

In [2]:
def update_rk2(f, t, y, h):
    return y + h*f(t+h/2, y + h/2*f(t, y))#y + h(1+h/2)y


def rk2(f, tini, tend, nt, yini):
    y = [yini]
    if(nt<=1):return y
    h = (tend-tini)/(nt-1)
    for i in range(1,nt):
        y += [update_rk2(f, tini + i*h, y[i-1], h)]
    return y



def f(t, y):
    return y

def sol(x):
    return np.exp(x)


N=4
n = [10**i for i in range (0,N+1)]
h = [1/ (10**i-1) for i in range (1,N+1)]
h_2 = [hi**2 for hi in h] # h étant inférieur à 1 et positif, h^2 <= h 

tini = 0
T = 1
x = np.linspace(tini, tini + T, 10**N)
exp = sol(x)


y = []
Max = []
for i in range(0,N+1):# calcul du max de l'erreur
    x= np.linspace(0, 1, 10**i)
    y = rk2(f, tini, T + tini, 10**i, 1)
    S = abs(y - sol(x))
    Max += [max(S)]
    

    

fig1 = figure(title="Superposition de la solution théorique et numérique :", width=500, height=350)
fig1.line(x, exp, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(x, y,  color="red", legend="Solution numérique")


fig2 = figure(title="Erreur de la Méthode de Runge-Kutta 2 :", width=500, height=350, x_axis_type="log", y_axis_type="log")
fig2.x(n, Max, color="blue", legend="Erreur")
fig2.line(n[1:], h_2,  color="red", legend="ordre 2")

show(row(fig1,fig2))

<center><b> La droite $y= h^2$ est bien parrallèle avec la droite passant par les points de l'erreur $\Rightarrow$ l'erreur est bien d'ordre 2. <b></center>

## La méthode RK4

**Implémenter la méthode RK4 et vérifier numériquement sur le même exemple qu'à la question précédente.**

In [3]:
def update_rk4(f, t, y, h):
    k1 =f(t, y)
    k2 =f(t + h/2, y +h/2*k1)
    k3 =f(t + h/2, y +h/2*k2)
    k4 =f(t + h, y +h*k3)
    
    return y + h/6*(k1+ 2*k2 + 2*k3 + k4)


def rk4(f, tini, tend, nt, yini):
    y = [yini]
    if(nt<=1):return y
    h = (tend-tini)/(nt-1)
    for i in range(1,nt):
        y += [update_rk4(f, tini + i*h, y[i-1], h)]
    return y



**Vérifier que cette méthode est d'ordre 4.**

In [4]:
N=4


n = [10**i for i in range (0,N+1)]
h = [1/ (10**i-1) for i in range (1,N+1)]
h_4 = [hi**4 for hi in h]


x = np.linspace(0, 1, 10**N)
exp = sol(x)


y_rk4 = []
Max_rk4 = []
for i in range(0,N+1):
    x= np.linspace(0, 1, 10**i)
    y_rk4 = rk4(f, 0, 1, 10**i, 1)
    S = abs(y_rk4 - sol(x))
    Max_rk4 += [max(S)]
    

    

fig1 = figure(title="Superposition de la solution théorique et numérique :", width=500, height=350)
fig1.line(x, exp, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(x, y_rk4,  color="red", legend="Solution numérique")


fig2 = figure(title="Erreur de la Méthode de Runge-Kutta 4 :", width=500, height=350, x_axis_type="log", y_axis_type="log")
fig2.x(n, Max_rk4, color="blue", legend="Erreur")
fig2.line(n[1:], h_4,  color="red", legend="ordre 4")

print("Les 3 dernières valeurs de l'erreur sont : ")
print("\r")
print(Max_rk4[2],Max_rk4[3], Max_rk4[4])

show(row(fig1,fig2))
## Le dernier point n'est pas en accord avec la droite d'ordre 4 => précision machine est atteinte

Les 3 dernières valeurs de l'erreur sont : 

2.3383694980338987e-10 2.6201263381153694e-14 1.2878587085651816e-14


<center><b> La droite $y= h^4$ est bien parrallèle avec la droite passant par tous les points de l'erreur sauf un seul $\Rightarrow$ l'erreur est bien d'ordre 4. </b></center><br>

***
 <center>On atteint la précision machine, les deux dernières valeurs de l'erreur sont autour de $10^{-14}$, pas loin de la précision machine de $10^{-16}$.</center>

## Tableau de Butcher 

**Bonus : Implémenter une méthode de Runge-Kutta générale explicite utilisant le tableau de Butcher. On implementera une fonction update_rk qui retourne $y_{n+1}$ étant données $f,t_n,y_n,h$ et les tableaux $A,B,C$ stockant respectivement les coefficients $(a_{ij})_{1 \leq i,j \leq p}$, $(b_j)_{j=1,...,p}$ et $(c_i)_{i=1,...,p}$ décrits dans le sujet de la PC.**

In [5]:
def update_rk(f, t, y, h, A, B, C):
    p = len(B)
    k = [f(t, y[-1])]
    for i in range (1, p):
        s = 0
        for j in range (0,i):
            s += A[i][j]*k[j]
        k += [f(t + C[i-1]* h, y[-1] + h*s)]
    ynp1 = y[-1]
    for i in range (0, p):
        ynp1 += h * B[i] * k[i]
    return ynp1


def rk(f, tini, tend, nt, yini, A, B, C):
    y = [yini]
    if(nt == 1): return y
    h = (tend-tini)/(nt-1)
    t = tini
    for i in range (1, nt):
        y += [update_rk(f, t, y, h, A, B, C)]
        t += h
    return y

**Bonus : Tester avec les tableaux de Butcher correspondant aux méthodes rk2 et rk4 déjà implémentés. Vérifier que vous obtenez les mêmes résultats sur le cas $f(t,y)=y$.**

In [6]:
## Verificcation de l'obtention des mêmes résultats

##Implémentation des matrices correspondantes à RK2
A_rk2 = [ [0, 0] , [1/2, 0] ]
B_rk2 = [0, 1]
C_rk2 = [1/2]

def f(t, y):
    return y

def sol(x):
    return np.exp(x)




N=4
n = [10**i for i in range (0,N+1)]
h = [1/ (10**i-1) for i in range (1,N+1)]
h_2 = [hi**2 for hi in h]


x = np.linspace(0, 1, 10**N)
exp = sol(x)


y = []
Max = []
for i in range(0,N+1):
    x= np.linspace(0, 1, 10**i)
    y = rk(f, 0, 1, 10**i, 1, A_rk2, B_rk2, C_rk2)
    S = abs(y - sol(x))
    Max += [max(S)]
    

    

fig1 = figure(title="Superposition de la solution théorique et numérique :", width=500, height=350)
fig1.line(x, exp, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(x, y,  color="red", legend="Solution numérique")


fig2 = figure(title="Erreur de la Méthode de Runge-Kutta 2 :", width=500, height=350, x_axis_type="log", y_axis_type="log")
fig2.x(n, Max, color="blue", legend="Erreur")
fig2.line(n[1:], h_2,  color="red", legend="ordre 2")

show(row(fig1,fig2))

<center><b> On obtient bien que la solution numérique coïncide avec la solution théorique comme pour RK2, et l'ordre est aussi réspecté $\Rightarrow$ Mêmes résultats que ci-dessus. </b></center>

***


In [7]:
N=4

##Implémentation des matrices correspondantes à RK4
A_rk4 = [ [0, 0, 0, 0] , [1/2, 0, 0, 0] , [0, 1/2, 0, 0] , [0, 0, 1, 0] ]
B_rk4 = [1/6, 1/3, 1/3, 1/6]
C_rk4 = [1/2, 1/2, 1]


n = [10**i for i in range (0,N+1)]
h = [1/ (10**i-1) for i in range (1,N+1)]
h_4 = [hi**4 for hi in h]


x = np.linspace(0, 1, 10**N)
exp = sol(x)


y_rk4 = []
Max_rk4 = []
for i in range(0,N+1):
    x= np.linspace(0, 1, 10**i)
    y_rk4 = rk(f, 0, 1, 10**i, 1, A_rk4, B_rk4, C_rk4)
    S = abs(y_rk4 - sol(x))
    Max_rk4 += [max(S)]
    
print(Max_rk4)
    

fig1 = figure(title="Superposition de la solution théorique et numérique :", width=500, height=350)
fig1.line(x, exp, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(x, y_rk4,  color="red", legend="Solution numérique")


fig2 = figure(title="Erreur de la Méthode de Runge-Kutta 4 :", width=500, height=350, x_axis_type="log", y_axis_type="log")
fig2.x(n, Max_rk4, color="blue", legend="Erreur")
fig2.line(n[1:], h_4,  color="red", legend="ordre 4")

show(row(fig1,fig2))
## Le dernier point n'est pas en accord avec la droite d'ordre 4 => précision machine est atteinte

[0.0, 3.14763266251461e-06, 2.338338411789209e-10, 2.2648549702353193e-14, 3.375077994860476e-14]


<center><b> On obtient bien que la solution numérique coïncide avec la solution théorique comme pour RK4, et l'ordre est aussi réspecté $\Rightarrow$ Mêmes résultats que ci-dessus. </b></center>

***


# Un système conservatif

On considère l'ocillateur harmonique :
\begin{equation}
\left\{
\begin{aligned}
q'&=p\\
p'&=-q.
\end{aligned}
\right.
\end{equation}

## Schéma d'Euler explicite et implicite

**Implémenter les schémas d'Euler explicite et implicite pour le système précédent.**

In [8]:
def euler_explicite(tini, tend, nt, qini, pini):
    y = [[qini],[pini]]
    if(nt<=1):return y
    h = (tend-tini)/(nt-1)
    for i in range(1,nt):
        y[0] += [y[0][i-1] + h *y[1][i-1]]#Equation pour q
        y[1] += [y[1][i-1] - h *y[0][i-1]]#Equation pour p
    return y


def euler_implicite(tini, tend, nt, qini, pini):
    y = [[qini],[pini]]
    if(nt<=1):return y
    h = (tend-tini)/(nt-1)
    for i in range(1,nt):
        y[0] += [(y[0][i-1] + h *y[1][i-1])/(1+h**2)]#Equation pour q
        y[1] += [(y[1][i-1] - h *y[0][i-1])/(1+h**2)]#Equation pour p
    return y
    

**Représenter les solutions $q_n$ et $p_n$ obtenues, ainsi que l'hamiltonien $\mathcal{H}(q_n,p_n)$ associé. On pourra tracer $q$ et $p$ en fonction du temps, mais aussi tracer la solution dans le plan de phase $(q,p)$. Commenter les résultats obtenus.**

In [9]:
##Ici, on observe le comportement du schéma explicite

nt=1000 #nombre de points
##On prend un interval de 10 sec
tini = 0
tend = 20   #explosion pour t = 200, l'approximation numérique explose. 
            #Attendu, puisque l'erreur sur l'énergie est positive et constante => explosion verrs l'infini


t = np.linspace(tini, tend, nt)
sol = np.cos(t)
dsol = -np.sin(t)

##On prend qini = 1 pour forcer que cos soit la solution => q' = -sin => pini  = 0
y_exp = euler_explicite(tini, tend, nt, 1, 0)

## On peut commencer à observer les divergences du système par rapport à la solution théorique
fig1 = figure(title="Superposition de la solution théorique et numérique explicite pour q(t) :", width=500, height=350)
fig1.line(t, sol, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(t, y_exp[0],  color="red", legend="Solution numérique")
fig1.legend.location = "bottom_right"


fig2 = figure(title="Superposition de la solution théorique et numérique explicite pour p(t) :", width=500, height=350)
fig2.line(t, dsol, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig2.line(t, y_exp[1],  color="red", legend="Solution numérique")


Hpq_exp = [(y_exp[0][i]**2 + y_exp[1][i]**2)/2 for i in range(0,nt)]
droite = [0.5]
h = (tend-tini)/(nt-1)
for i in range (1,nt):#le resultat théorique nous indique que Hpq_exp(n+1)=(1+h^2)*Hpq_exp(n)
    droite += [(1+h**2)*droite[i-1]]

# pour
# x = cos(t)
# y = -sin(t)
# On obtient bien un cercle de centre I(0,0) et de rayon 1
fig3 = figure(title="Plan de phase :", width=500, height=350)
fig3.line(y_exp[0], y_exp[1],  color="red", legend="Solution numérique")
fig3.line(sol, dsol,  line_width=2, color="blue", legend="Comportement théorique", line_dash= [5,10])


## Elle n'est pas constante et augmente => validation des calculs théoriques
fig4 = figure(title="Hamiltonien explicite :", width=500, height=350)
fig4.line(t, Hpq_exp,  color="red", legend="Solution numérique")
fig4.line(t, droite,  line_width=2, color="blue", legend="Comportement théorique du Hamiltonien numérique", line_dash= [5,10])
fig4.legend.location = "top_left"


show(row(fig1, fig2))
show(row(fig3, fig4))

### Commentaires :
<ol>
<li>Le Hamiltonien est croissant en fonction du temps, c'est une droite à pente positive $\Rightarrow$ on retrouve bien le comportement théorique</li>  
<li> Comme l'enérgie croit, il est normal que $q_{max}$ et $p_{max}$ augmentent aussi (figure 1 et 2). Ce qui explique la divergence avec le modèle théorique.</li>
<li>Le portait de phase est aussi cohérent, le rayon du cercle augmente parce que $R(t) = \sqrt{q^2(t) + p^2(t)}  $.</li>  
</ol>

In [10]:
##Ici, on observe le comportement du schéma explicite

nt=1000 #nombre de points
##On prend un interval de 10 sec
tini = 0
tend = 20  #extinction pour t = 200, l'approximation numérique tend vers 0. 
            #Attendu, puisque l'erreur sur l'énergie est negative => l'energie étant positive, s'annule.

t = np.linspace(tini, tend, nt)
sol = np.cos(t)
dsol = -np.sin(t)

##On prend qini = 1 pour forcer que cos soit la solution => q' = -sin => pini  = 0
y_imp = euler_implicite(tini, tend, nt, 1, 0)

## On peut commencer à observer les divergences du système par rapport à la solution théorique
fig1 = figure(title="Superposition de la solution théorique et numérique implicite pour q(t) :", width=500, height=350)
fig1.line(t, sol, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(t, y_imp[0],  color="red", legend="Solution numérique")
fig1.legend.location = "bottom_right"


fig2 = figure(title="Superposition de la solution théorique et numérique implicite pour p(t) :", width=500, height=350)
fig2.line(t, dsol, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig2.line(t, y_imp[1],  color="red", legend="Solution numérique")
fig2.legend.location = "bottom_right"


Hpq_imp = [(y_imp[0][i]**2 + y_imp[1][i]**2)/2 for i in range(0,nt)]
droite = [0.5]
h = (tend-tini)/(nt-1)
for i in range (1,nt):#le resultat théorique nous indique que Hpq_exp(n+1)=(1+h^2)*Hpq_exp(n)
    droite += [droite[i-1]/(1+h**2)]

# pour
# x = cos(t)
# y = -sin(t)
# On obtient bien un cercle de centre I(0,0) et de rayon 1
fig3 = figure(title="Plan de phase :", width=500, height=350)
fig3.line(y_imp[0], y_imp[1],  color="red", legend="Solution numérique")
fig3.line(sol, dsol,  line_width=2, color="blue", legend="Comportement théorique", line_dash= [5,10])


## Elle n'est pas constante et diminue => validation des calculs théoriques
fig4 = figure(title="Hamiltonien implicite :", width=500, height=350)
fig4.line(t, Hpq_imp,  color="red", legend="Solution numérique")
fig4.line(t, droite,  line_width=2, color="blue", legend="Comportement théorique du Hamiltonien numérique", line_dash= [5,10])
fig4.legend.location = "top_left"


show(row(fig1, fig2))
show(row(fig3, fig4))

### Commentaires :
<ol>
<li>Le Hamiltonien est décroissant en fonction du temps, c'est une droite à pente négative $\Rightarrow$ on retrouve bien le comportement théorique</li>  
<li> Comme l'enérgie décroit, il est normal que $q_{max}$ et $p_{max}$ diminuent aussi (figure 1 et 2). Ce qui explique la divergence avec le modèle théorique.</li>
<li>Le portait de phase est aussi cohérent, le rayon du cercle diminue parce que $R(t) = \sqrt{p^2(t) + q^2(t)} $.</li>  
</ol>

## Schéma d'ordre élevé

**Appliquer le schéma RK2 (décrit dans l'exercice précédent) au système de l'ocillateur harmonique. Comparer les résultats $p_n$, $q_n$ et $\mathcal{H}(q_n,p_n)$ avec ceux obtenus à la question précédente. Commenter les résultats obtenus.**

##### Schéma explicite de RK2 :
Par calcul on obtient :
\begin{equation}
\left\{
\begin{aligned}
q_{n+1}&=q_n + h(p_n - \cfrac{h}{2}q_n)\\
p_{n+1}&=p_n - h(q_n + \cfrac{h}{2}p_n).
\end{aligned}
\right.
\end{equation}

In [21]:
##On code le schéma explicite de RK2

def rk2_mod_exp(tini, tend, nt, yini):
    y = [[yini[0]],[yini[1]]]
    if(nt<=1):return y
    h = (tend-tini)/(nt-1)
    for i in range(1,nt):
        y[0] += [y[0][i-1] + h *(y[1][i-1] - (h/2)*y[0][i-1])]#Equation pour q
        y[1] += [y[1][i-1] - h *(y[0][i-1] + (h/2)*y[1][i-1])]#Equation pour p
    return y  

nt=1000 #nombre de points
##On prend un interval de 10 sec
tini = 0
tend = 20   #explosion pour t = 200, l'approximation numérique explose. 
            #Attendu, puisque l'erreur sur l'énergie est positive et constante => explosion verrs l'infini


t = np.linspace(tini, tend, nt)
sol = np.cos(t)
dsol = -np.sin(t)

##On prend qini = 1 pour forcer que cos soit la solution => q' = -sin => pini  = 0
y_exp = euler_explicite(tini, tend, nt, 1, 0)
y_rk2_exp = rk2_mod_exp(tini, tend, nt, [1, 0])

fig1 = figure(title="Superposition de la solution d'Euler et RK2 explicite pour q(t) :", width=500, height=350)
fig1.line(t, y_exp[0], line_width=2, color="blue", legend="Solution euler explicite", line_dash= [5,10]) 
fig1.line(t, y_rk2_exp[0],  color="red", legend="Solution RK2 explicite")
fig1.legend.location = "bottom_right"


fig2 = figure(title="Superposition de la solution d'Euler et RK2 explicite pour p(t) :", width=500, height=350)
fig2.line(t, y_exp[1], line_width=2, color="blue", legend="Solution euler explicite", line_dash= [5,10]) 
fig2.line(t, y_rk2_exp[1],  color="red", legend="Solution RK2 explicite")


Hpq_exp_rk2 = [(y_rk2_exp[0][i]**2 + y_rk2_exp[1][i]**2)/2 for i in range(0,nt)]
droite = [0.5]
h = (tend-tini)/(nt-1)
for i in range (1,nt):#le resultat théorique nous indique que Hpq_exp(n+1)=(1+h^2)*Hpq_exp(n)
    droite += [(1+h**2)*droite[i-1]]

# pour
# x = cos(t)
# y = -sin(t)
# On obtient bien un cercle de centre I(0,0) et de rayon 1
fig3 = figure(title="Plan de phase :", width=500, height=350)
fig3.line(y_rk2_exp[0], y_rk2_exp[1],  color="red", legend="Solution RK2 explicite")
fig3.line(y_exp[0], y_exp[1], line_width=2, color="blue", legend="Solution euler explicite", line_dash= [5,10]) 


## Elle n'est pas constante et augmente => validation des calculs théoriques
fig4 = figure(title="Hamiltonien RK2 explicite :", width=500, height=350)
fig4.line(t, Hpq_exp_rk2,  color="red", legend="Hamiltonien RK2 explicite")
fig4.line(t, droite,  line_width=2,  color="blue", legend="Hamiltonien explicite d'Euler", line_dash= [5,10])
fig4.legend.location = "top_left"


show(row(fig1, fig2))
show(row(fig3, fig4))

##### Schéma implicite de RK2 :
Par calcul on obtient :
\begin{equation}
\left\{
\begin{aligned}
q_{n+1}&=\cfrac{hp_n + q_n\left(1 + \cfrac{h}{2}^2\right)}{h^2+\left(1+\cfrac{h^2}{2}\right)^2}\\
p_{n+1}&=\cfrac{-hq_n + p_n\left(1 + \cfrac{h}{2}^2\right)}{h^2+\left(1+\cfrac{h^2}{2}\right)^2}.
\end{aligned}
\right.
\end{equation}

### Remarques :

<ol>
<li> Une énorme baisse dans la divergence quand t augmente. </li>
<li> La différence majeur est que la pente du Hamiltonien est beaucoup moin raide, le Hamiltonien est presque constant.</li>
<li> Mais il n'est pas parfaitement contant comme celui du schéma symplectique. On observe quand même la divergence quant t est très grand.</li>
</ol>

## Schéma symplectique

Considérez le schéma suivant :

\begin{equation*}
  \label{schema_symp}
  y_{n+1} = y_n + h f\left(t_n+\frac{h}{2},\ \frac{y_n+y_{n+1}}{2}\right).
\end{equation*}
<br>
***

Par calcul on obtient :
\begin{equation}
\left\{
\begin{aligned}
q_{n+1}&=\cfrac{\left(1- \cfrac{h^2}{4}\right)q_n +hp_n }{\left(1 + \cfrac{h^2}{4}\right)} \\
p_{n+1}&=\cfrac{-hq_n + \left(1- \cfrac{h^2}{4}\right)p_n }{\left(1 + \cfrac{h^2}{4}\right)} .
\end{aligned}
\right.
\end{equation}<br><br>

***

In [12]:
def pt_milieu_symplectique(tini, tend, nt, yini):
    y = [[yini[0]],[yini[1]]]
    if(nt<=1):return y
    h = (tend-tini)/(nt-1)
    for i in range(1,nt):
    #On utilise (1-h/2)*(1+h/2) = (1-h**2/4) parce que numériquement (1-h/2)*(1+h/2) est plus précis surtout quand h petit 
    #On utilise (1- 1j*h/2)*(1+ 1j*h/2).real = (1+h**2/4) parce que c'est aussi numériquement quand h petit
    
        y[0] += [ ((1-h/2)*(1+h/2)*y[0][i-1] +          h*y[1][i-1])/((1- 1j*h/2)*(1+ 1j*h/2)).real]#Equation pour q
        y[1] += [ (      -h * y[0][i-1] + (1-h/2)*(1+h/2)*y[1][i-1])/((1- 1j*h/2)*(1+ 1j*h/2)).real]#Equation pour p
        
    return y  


**Observer numériquement son ordre, ainsi que l'évolution de l'hamiltonien, et commenter les résultats obtenus.**

In [20]:
nt=1000 #nombre de points
##On prend un interval de 10 sec
tini = 0
tend = 20 


t = np.linspace(tini, tend, nt)
sol = np.cos(t)
dsol = -np.sin(t)

##On prend qini = 1 pour forcer que cos soit la solution => q' = -sin => pini  = 0
y_symp = pt_milieu_symplectique(tini, tend, nt, [1, 0])

N = 4
n = [10**i for i in range (0,N+1)]
h = [1/ (10**i-1) for i in range (1,N+1)]
h_2 = [hi**2 for hi in h]
Max_symp = []
for ni in n:
    x = np.linspace(tini, tend, ni)
    y = pt_milieu_symplectique(tini, tend, ni, [1, 0])
    S = max([max(abs(y[0] - np.cos(x))), max(abs(y[1] + np.sin(x)))])
    Max_symp += [S]



fig1 = figure(title="Superposition de la solution théorique et symplectique q(t) :", width=500, height=350)
fig1.line(t, sol, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig1.line(t, y_symp[0],  color="red", legend="Solution numérique")
fig1.legend.location = "bottom_right"


fig2 = figure(title="Superposition de la solution théorique et symplectique pour p(t) :", width=500, height=350)
fig2.line(t, dsol, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig2.line(t, y_symp[1],  color="red", legend="Solution numérique")


Hpq_symp = [(y_symp[0][i]**2 + y_symp[1][i]**2)/2 for i in range(0,nt)]
droite = [0.5 for i in range (0,nt)]

# pour
# x = cos(t)
# y = -sin(t)
# On obtient bien un cercle de centre I(0,0) et de rayon 1
fig3 = figure(title="Plan de phase :", width=500, height=350)
fig3.line(y_symp[0], y_symp[1],  color="red", legend="Solution symplectique")
fig3.line(sol, dsol,  line_width=2, color="blue", legend="Comportement théorique", line_dash= [5,10])


## Elle n'est pas constante et augmente => validation des calculs théoriques
fig4 = figure(title="Hamiltonien symplectique :", width=500, height=350)
fig4.line(t, Hpq_symp,  color="red", legend="Solution numérique")
fig4.line(t, droite,  line_width=2, color="blue", legend="Comportement Théorique", line_dash= [5,10])
fig4.legend.location = "top_left"

fig5 = figure(title="Erreur du schéma Symplectique :", width=500, height=350, x_axis_type="log", y_axis_type="log")
fig5.x(n, Max_symp, color="blue", legend="Erreur")
fig5.line(n[1:], h_2,  color="red", legend="ordre 2")


show(row(fig1, fig2))
show(row(fig3, fig4))
show(fig5)

##On observe bien la conservation du hamiltonien comme prévu même s'il augmente que très légèrement.
##L'energie étant conservé, le portrait de phase colle bien avec celui d'un oscillateur harmonique soumis à aucun frottement

##### Commentaires :

<ol>
<li>On observe bien la conservation de l'hamiltonien comme prévu même s'il augmente que très légèrement.</li>
<li>L'énergie étant conservée, le portrait de phase colle bien avec celui d'un oscillateur harmonique soumis à aucun frottement $\Rightarrow$ pas de divergence grossière comme précédemment avec la méthode d'euler!</li>
<li>En plus, l'ordre de convergence de l'erreur du schéma symplectique est d'ordre 2. C'est le même ordre que celui de RK2, tout en conservant l'énergie. D'ailleur cette méthode est plus robuste que RK2, lorsque t augmente beaucoup elle reste plus stable que RK2 $\Rightarrow$ C'est le meilleur schéma pour ce problème.</li>
</ol>

